In [0]:
import nltk
nltk.download('stopwords')
  
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def read_article(filedata):
    # file = open(file_name, "r")
    # filedata = file.readlines()
    article = filedata.split(".")
    sentences = []
    for sentence in article:
        # print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name):
    stop_words = stopwords.words('english')
    summarize_text = []
    temp_text = []
    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank_numpy(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True) 
    # print("Indexes of top ranked_sentence order are ", ranked_sentence)    
    finalText = ""
    for i in range(0,len(ranked_sentence)):
      temp_text.append(" ".join(ranked_sentence[i][1]))
      # summarize_text.append(" ".join(ranked_sentence[i][1]))
      if (len(". ".join(temp_text)) < 127):
          summarize_text.append(" ".join(ranked_sentence[i][1]))
          continue;
      else:
          break;

    # Step 5 - Offcourse, output the summarize texr
    finalSummarizeText = ". ".join(summarize_text)
    # print("Summarize Text: \n", finalSummarizeText)
    # print("length of summarized text: ", len(finalSummarizeText))
    return finalSummarizeText


In [0]:
import pandas as pd
df2 = pd.read_csv('drive/My Drive/Colab Notebooks/se-3.csv')


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
set1 = df2[:500]
set2 = df2[-500:]
set1 = set1.append(set2, ignore_index = True) 

In [0]:
import re

set1['question1_processed'] = set1['question1'].map(lambda x: re.sub('[,]|comments|answers|questions|question|answer|tags|comment|tag|post|community|ell', '', str(x)))
set1['question1_processed'] = set1['question1_processed'].map(lambda x: re.sub('[\!\?]', '.', str(x)))
set1['question2_processed'] = set1['question2'].map(lambda x: re.sub('[,]|comments|answers|questions|question|answer|tags|comment|tag|post|community|ell', '', str(x)))
set1['question2_processed'] = set1['question2_processed'].map(lambda x: re.sub('[\!\?]', '.', str(x)))
set1['question1_processed'] = set1['question1_processed'].map(lambda x: x.lower())
set1['question2_processed'] = set1['question2_processed'].map(lambda x: x.lower())

In [0]:
# index = 5238
for i in range(0, 1000):
  if i != 0 and i % 100 == 0:
     print("{:,} sentences embedded.".format(i), flush=True)
  # set1['id'][i] = index
  # index = index+1
  set1['question1'][i] = set1['question1_processed'][i] = set1['question1_processed'][i].replace("\r","").replace("\n","")
  set1['question2'][i] = set1['question2_processed'][i] = set1['question2_processed'][i].replace("\r","").replace("\n","")
  # print(set1['question1'][i])
  try:
  # print("Sentence: ", df2['question1_processed'][i])
    if len(set1['question1_processed'][i]) > 127:
        summary = generate_summary(set1['question1_processed'][i])
        # print("Summary1: ", summary);
        if not summary:
            summary = set1['question1_processed'][i]
            # print("Summary2: ", summary);
    else:
        summary = set1['question1_processed'][i]
        # print("Summary3: ", summary);
  
    set1['question1'][i] = summary

    if len(set1['question2_processed'][i]) > 127:
        summary = generate_summary(set1['question2_processed'][i])
        # print("Summary1: ", summary);
        if not summary:
            summary = set1['question2_processed'][i]
            # print("Summary2: ", summary);
    else:
        summary = set1['question2_processed'][i]
        # print("Summary3: ", summary);
    
    set1['question2'][i] = summary
  except:
    continue;

# print(set1)
# set1.set_index('id', inplace=True)
# print (set1)
# with open('drive/My Drive/askUbuntuFinal.csv', 'a') as f:
#     set1.to_csv(f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is 

100 sentences embedded.
200 sentences embedded.


/usr/local/lib/python3.6/dist-packages/nltk/cluster/util.py:133: RuntimeWarning: invalid value encountered in true_divide
  sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))


300 sentences embedded.
400 sentences embedded.
500 sentences embedded.
600 sentences embedded.
700 sentences embedded.
800 sentences embedded.
900 sentences embedded.


In [0]:
# set1.set_index('id', inplace=True)
from sklearn.utils import shuffle
set1 = shuffle(set1)
# with open('drive/My Drive/politicsFinal.csv', 'a') as f:
#     set1.to_csv(f)

In [0]:
print(set1)
with open('drive/My Drive/stackOverflowP.csv', 'w') as f:
    set1.to_csv(f)

     Unnamed: 0  ...                                question2_processed
72           72  ...  can i automatically increment the file build v...
174         174  ...  how can i profile c++ code running on linux.i ...
676       41434  ...  asp.net mvc example of editing multiple child ...
618       41376  ...  what's the best way to let a user pick a subdi...
873       41631  ...  alternative to itoa() for converting integer t...
..          ...  ...                                                ...
12           12  ...  storing images in db - yea or nay.so i'm using...
620       41378  ...  c++ wrapper with same name.how can i do a wrap...
246         246  ...  how can i modify .xfdl files. (update #1)the ....
470         470  ...  what are differences between assemblyversion a...
344         344  ...  what are 'closures' in .net.what is a closure....

[1000 rows x 9 columns]


In [0]:
set1["question1"][6]

'  my  is:what is the best way to deal with storing images in postgres'

In [0]:
for i in range(0,10):
  print("i", set1["question1"][i])

i calculate relative time in c#given a specific datetime value how do i display relative time like:2 hours ago3 days agoa month ago
i  so. . .  should constants be capitalized.  or is that an archaic practice. should i capitalize my constants
i jeff finished his  talking about this but i don't catch the idea. so why do you think this is a bad coding style
i determine a user's timezoneis there a standard way for a web server to be able to determine a user's timezone within a web page. perhaps from an http header or part of the user-agent string.
i binary data in mysqlhow do i store binary data in mysql.
i generally how bad of a performance hit is storing a file in a database (specifically mssql) as opposed to the file system
i   my  is:what is the best way to deal with storing images in postgres
i   it is intended to manage a few megabytes of information but also manage scanned images in full resolution
i  this  is specifically about the pros/cons of using a db or fs for a small number 